# Análisis Exploratorio de Datos (EDA)
## Proyecto Final: Predicción de Condiciones para Vuelo a Vela

**Objetivo:** Entender los datos de vuelos y meteorología antes de modelar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Carga de Datos

In [ ]:
# Cargar dataset de desarrollo
df = pd.read_csv('../data/processed/vuelos_dev.csv')

print(f"Shape del dataset: {df.shape}")
print(f"\nPrimeras filas:")
df.head()

In [ ]:
# Información general
df.info()

## 2. Variables Target

Analizamos las variables que queremos predecir

In [ ]:
# Variables target
targets = ['altura_max_m', 'ganancia_altura_m', 'duracion_min', 'distancia_km', 'calidad_dia']

print("Estadísticos descriptivos de targets:")
df[targets[:-1]].describe()

In [ ]:
# Distribución de calidad_dia
print("Distribución de calidad del día:")
print(df['calidad_dia'].value_counts().sort_index())
print(f"\nPorcentajes:")
print(df['calidad_dia'].value_counts(normalize=True).sort_index() * 100)

In [ ]:
# Visualización de targets
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Altura máxima
axes[0, 0].hist(df['altura_max_m'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Altura Máxima (m)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución de Altura Máxima')
axes[0, 0].axvline(df['altura_max_m'].mean(), color='red', linestyle='--', label=f'Media: {df["altura_max_m"].mean():.0f}m')
axes[0, 0].legend()

# Duración
axes[0, 1].hist(df['duracion_min'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Duración (min)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Duración')
axes[0, 1].axvline(df['duracion_min'].mean(), color='red', linestyle='--', label=f'Media: {df["duracion_min"].mean():.0f}min')
axes[0, 1].legend()

# Distancia
axes[1, 0].hist(df['distancia_km'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 0].set_xlabel('Distancia (km)')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].set_title('Distribución de Distancia')
axes[1, 0].axvline(df['distancia_km'].mean(), color='red', linestyle='--', label=f'Media: {df["distancia_km"].mean():.0f}km')
axes[1, 0].legend()

# Calidad del día
calidad_counts = df['calidad_dia'].value_counts().sort_index()
axes[1, 1].bar(calidad_counts.index, calidad_counts.values, edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Calidad del Día')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].set_title('Distribución de Calidad del Día')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3. Variables Meteorológicas

Exploramos las features que usaremos para predecir

In [ ]:
# Identificar features meteorológicas
features_meteo = [col for col in df.columns if col not in targets + 
                  ['flight_id', 'fecha', 'pilot', 'glider', 'mes', 'dia_año', 'fecha_dt']]

print(f"Features meteorológicas disponibles: {len(features_meteo)}")
print("\nLista de features:")
for i, feat in enumerate(features_meteo, 1):
    print(f"{i:2d}. {feat}")

In [ ]:
# Estadísticos de features meteorológicas
print("Estadísticos de features meteorológicas:")
df[features_meteo].describe().T

In [ ]:
# Verificar valores faltantes
print("Valores faltantes por columna:")
missing = df[features_meteo].isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if len(missing) > 0:
    print(missing)
else:
    print("No hay valores faltantes en features meteorológicas ✓")

In [ ]:
# Visualizar distribuciones de features clave
features_clave = [
    'temp_2m_max',
    'solar_rad_max',
    'wind_speed_mean',
    'cape_max',
    'cloud_cover_mean',
    'boundary_layer_height_max'
]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, feat in enumerate(features_clave):
    axes[i].hist(df[feat], bins=30, edgecolor='black', alpha=0.7)
    axes[i].set_xlabel(feat)
    axes[i].set_ylabel('Frecuencia')
    axes[i].set_title(f'Distribución de {feat}')
    axes[i].axvline(df[feat].mean(), color='red', linestyle='--', label=f'Media: {df[feat].mean():.2f}')
    axes[i].legend()

plt.tight_layout()
plt.show()

## 4. Correlaciones

Analizar correlaciones entre features y targets

In [ ]:
# Matriz de correlación con targets numéricos
targets_numericos = ['altura_max_m', 'duracion_min', 'distancia_km']
correlaciones = df[features_clave + targets_numericos].corr()

# Visualizar
plt.figure(figsize=(12, 10))
sns.heatmap(correlaciones, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Matriz de Correlación: Features vs Targets')
plt.tight_layout()
plt.show()

In [ ]:
# Correlaciones específicas con altura_max_m
corr_altura = df[features_meteo + ['altura_max_m']].corr()['altura_max_m'].sort_values(ascending=False)

print("Top 10 features más correlacionadas con altura_max_m:")
print(corr_altura.head(11)[1:])  # Excluir la auto-correlación

print("\nTop 10 features menos correlacionadas (negativas):")
print(corr_altura.tail(10))

In [ ]:
# Visualizar correlaciones más fuertes
top_features = corr_altura.head(7)[1:].index

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, feat in enumerate(top_features[:6]):
    axes[i].scatter(df[feat], df['altura_max_m'], alpha=0.5)
    axes[i].set_xlabel(feat)
    axes[i].set_ylabel('Altura Máxima (m)')
    axes[i].set_title(f'{feat} vs Altura Máxima\nCorr: {corr_altura[feat]:.3f}')
    
    # Línea de tendencia
    z = np.polyfit(df[feat], df['altura_max_m'], 1)
    p = np.poly1d(z)
    axes[i].plot(df[feat], p(df[feat]), "r--", alpha=0.8)

plt.tight_layout()
plt.show()

## 5. Análisis por Calidad del Día

Comparar features según la calidad del día

In [ ]:
# Boxplots de features clave por calidad
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, feat in enumerate(features_clave):
    df.boxplot(column=feat, by='calidad_dia', ax=axes[i])
    axes[i].set_xlabel('Calidad del Día')
    axes[i].set_ylabel(feat)
    axes[i].set_title(f'{feat} por Calidad')
    plt.sca(axes[i])
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 6. Detección de Outliers

In [ ]:
# Método IQR para detección de outliers en targets
def detectar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[columna] < lower_bound) | (df[columna] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

print("Análisis de outliers en targets:\n")
for target in targets_numericos:
    outliers, lb, ub = detectar_outliers_iqr(df, target)
    print(f"{target}:")
    print(f"  Outliers detectados: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
    print(f"  Rango normal: [{lb:.1f}, {ub:.1f}]")
    print()

## 7. Análisis Temporal

In [ ]:
# Convertir fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Altura promedio por mes
altura_por_mes = df.groupby('mes')['altura_max_m'].agg(['mean', 'std', 'count'])

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Altura promedio por mes
axes[0].bar(altura_por_mes.index, altura_por_mes['mean'], yerr=altura_por_mes['std'], 
            capsize=5, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Mes')
axes[0].set_ylabel('Altura Máxima Media (m)')
axes[0].set_title('Altura Máxima Promedio por Mes')
axes[0].grid(axis='y', alpha=0.3)

# Cantidad de vuelos por mes
axes[1].bar(altura_por_mes.index, altura_por_mes['count'], alpha=0.7, 
            edgecolor='black', color='orange')
axes[1].set_xlabel('Mes')
axes[1].set_ylabel('Cantidad de Vuelos')
axes[1].set_title('Cantidad de Vuelos por Mes')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Conclusiones del EDA

**Escribe aquí tus observaciones:**

1. Distribución de targets:
   - ...

2. Features más importantes:
   - ...

3. Outliers detectados:
   - ...

4. Patrones temporales:
   - ...

5. Próximos pasos:
   - Preprocesamiento necesario
   - Feature engineering
   - Selección de features

## 9. Guardar Insights

Guardar información relevante para el modelado

In [ ]:
# Guardar lista de features importantes
top_features_lista = corr_altura.head(11)[1:].index.tolist()

import json

insights = {
    'features_meteo': features_meteo,
    'top_features_correlacion': top_features_lista,
    'targets_numericos': targets_numericos,
    'estadisticos_targets': df[targets_numericos].describe().to_dict()
}

with open('../data/processed/eda_insights.json', 'w') as f:
    json.dump(insights, f, indent=2)

print("✓ Insights guardados en: data/processed/eda_insights.json")